In [ ]:
%load_ext line_profiler

In [ ]:
import numpy as np
import os
import glob 
import matplotlib.pyplot as plt

In [ ]:
all_observation_folders = glob.glob(
    '.../kasanka-bats/processed/deep-learning/16Nov/*')
observation_folders = []
for folder in all_observation_folders:
    if not os.path.exists(os.path.join(folder, 'raw_tracks.npy')):
        observation_folders.append(folder)
print('Folders to use:')
print(*observation_folders, sep='\n')

In [ ]:
observation_folders = all_observation_folders
print(len(observation_folders))

In [ ]:
def combine_overlapping_tracks(observation_folder, first_group=0, last_group=None, save=False):
    track_files = glob.glob(os.path.join(observation_folder, 'first_frame*.npy'))
    track_files = sorted(track_files, key=lambda f: int(f.split('_')[-6]))

    track_groups = []
    for file in track_files:
        track_groups.append(np.load(file, allow_pickle=True))
        
        
    for track_file in track_files:
        print(os.path.basename(track_file))
        
    first_overlap_frames = [int(f.split('_')[-6]) for f in track_files[1:]]
    first_overlap_frames.append(None)
    print(first_overlap_frames)
        
    all_tracks = []

    total_tracks = 0
    for group_ind, track_group in enumerate(track_groups[first_group:last_group]):
        if group_ind >= len(track_groups) -1:
            for track in track_group:
                if type(track['track']) == list:
                    track['track'] = np.stack(track['track'])
                    track['pos_index'] = np.stack(track['pos_index'])
                    if 'size' in track:
                        track['size'] = np.stack(track['size'])
                all_tracks.append(track)
            total_tracks += len(track_group)
            break

        for track_ind, track in enumerate(track_group):
            if track['first_frame'] < first_overlap_frames[first_group + group_ind]:
                all_tracks.append(track)


    all_tracks_file = os.path.join(observation_folder, 'raw_tracks.npy')
    if save:
        np.save(all_tracks_file, all_tracks)
        print('saved')

In [ ]:
for folder in observation_folders:
    combine_overlapping_tracks(folder, save=True)

In [ ]:
def combine_overlapping_tracks(observation_folder, first_group=0, last_group=None, save=False):
    track_files = glob.glob(os.path.join(observation_folder, 'first_frame*.npy'))
    track_files = sorted(track_files, key=lambda f: int(f.split('_')[-6]))

    track_groups = []
    for file in track_files:
        track_groups.append(np.load(file, allow_pickle=True))
        
        
    for track_file in track_files:
        print(os.path.basename(track_file))
        
    first_overlap_frames = [int(f.split('_')[-6]) for f in track_files[1:]]
    first_overlap_frames.append(None)
    print(first_overlap_frames)
        
    all_tracks = []
    track_pairs = []
    not_found = []
    not_found_no_ff = []
    found = 0
    count = 0
    
    num_overlappingg = 0

    issue = 0

    total_tracks = 0
    for group_ind, track_group in enumerate(track_groups[first_group:last_group]):
        if group_ind >= len(track_groups) -1:
            for track in track_group:
                if type(track['track']) == list:
                    track['track'] = np.stack(track['track'])
                    track['pos_index'] = np.stack(track['pos_index'])
                    if 'size' in track:
                        track['size'] = np.stack(track['size'])
                all_tracks.append(track)
            total_tracks += len(track_group)
            break
        total_tracks += len(track_group)
        
        sorted_tracks = sorted(track_group, key=lambda t: t['first_frame'])
        
        alt_track_group = track_groups[first_group + group_ind + 1]
        alt_sorted_tracks, ff_val, ff_map, ff_numtracks  = sort_alt_tracks(alt_track_group)
        ff_ind = 0
        for track_ind, track in enumerate(sorted_tracks):
            if type(track['track']) == list:
                issue += 1
            if track['first_frame'] < first_overlap_frames[first_group + group_ind]:
                all_tracks.append(track)
            else:
                num_overlapping += 1
                if track['first_frame'] > ff_val[ff_ind]:
                    while track['first_frame'] > ff_val[ff_ind]:
                        ff_ind += 1
                        
                if track['first_frame'] < ff_val[ff_ind]:
                    all_tracks.append(track)
                    not_found_no_ff.append(track)
                    continue
                    
                if track['first_frame'] == ff_val[ff_ind]:
                    searching = True
                    for alt_track in alt_sorted_tracks[ff_map[ff_ind]:ff_map[ff_ind]+ff_numtracks[ff_ind]]:
                        if np.all(alt_track['track'][0] == track['track'][0]):
                            count += 1
                            track_pairs.append([track, alt_track])
                            searching = False
                            break 
                    if searching:
                        all_tracks.append(track)
                        not_found.append(track)


    print(f'{len(track_pairs)} track pairs. {len(not_found)} unmatched. {len(not_found_no_ff)} not sharing a first_frame')
    
    return not_found, not_found_no_ff

    all_tracks_file = os.path.join(observation_folder, 'raw_tracks.npy')
    if save:
        np.save(all_tracks_file, all_tracks)
        print('saved')
    print(f'{issue} issue tracks')
    

In [ ]:
def sort_alt_tracks(track_group):
    sorted_tracks = sorted(track_group, key=lambda t: t['first_frame'])
    # For each first frame ind, give the ind of the first track 
    # with that first  frame value
    lowest_firstframe = sorted_tracks[0]['first_frame']
    highest_firstframe = sorted_tracks[-1]['first_frame']
    firstframe_map = []
    firstframe_numtracks = []
    firstframe_val = []
    curr_track_ind = 0
    for firstframe in range(lowest_firstframe, highest_firstframe):
        assert sorted_tracks[curr_track_ind]['first_frame'] >= firstframe, "Check if first_overlap_frame is correct"
        if sorted_tracks[curr_track_ind]['first_frame'] == firstframe:
            firstframe_map.append(curr_track_ind)
            firstframe_val.append(firstframe)
            num_tracks = 0
            while sorted_tracks[curr_track_ind]['first_frame'] == firstframe:
                # skip all tracks with same first frame
                curr_track_ind += 1
                num_tracks += 1
            firstframe_numtracks.append(num_tracks)
            
    return sorted_tracks, firstframe_val, firstframe_map, firstframe_numtracks
                
    

In [ ]:
observation_folder = observation_folders[0]
track_files = glob.glob(os.path.join(observation_folder, 'first_frame*.npy'))
track_files = sorted(track_files, key=lambda f: int(f.split('_')[-6]))
for track_file in track_files:
    print(os.path.basename(track_file))

track_groups = []
for file in track_files[0:]:
    track_groups.append(np.load(file, allow_pickle=True))  

In [ ]:
# for val, track_ind, num_tracks in zip(ff_val, ff_map, ff_numtracks):
#     ffs = []
#     for ind in range(track_ind, track_ind + num_tracks):
#         ffs.append(sorted_tracks[ind]['first_frame'])
#     print(f'val: {val}', ffs)

In [ ]:
# not_found[0]

In [ ]:
not_found, not_found_no_ff = combine_overlapping_tracks(observation_folders[0], first_group=0, last_group=7, save=False)

In [ ]:
%lprun -f combine_overlapping_tracks combine_overlapping_tracks(observation_folders[0], first_group=0, last_group=None, save=False)

In [ ]:
for observation_folder in observation_folders:
    

In [ ]:
track_files

In [ ]:
for file in track_files:
    try:
        print(os.path.basename(file))
        track_groups = np.load(file, allow_pickle=True) 
    except:
        print('failed.')

In [ ]:
track_groups = []
for file in track_files[0:]:
    track_groups.append(np.load(file, allow_pickle=True))

In [ ]:
for track_group in track_groups:
    print(len(track_group))

In [ ]:

print(first_overlap_frames)

In [ ]:
track_group[0]

In [ ]:
issue = 0
track_issue = 0

first_group = 0
last_group = None

for track_group in track_groups:
    for i, alt_track in enumerate(track_group):
        if type(alt_track['track']) == list:
            issue += 1
            
for group_ind, track_group in enumerate(track_groups[first_group:last_group]):
#     if group_ind >= len(track_groups) -1:
#         break
    for track_ind, track in enumerate(track_group):
        if type(track['track']) == list:
            track_issue += 1
print(issue, track_issue)

In [ ]:
all_tracks = []
track_pairs = []
not_found = []
found = 0
count = 0

issue=0
first_group = 0
last_group = None
total_tracks = 0
for group_ind, track_group in enumerate(track_groups[first_group:last_group]):
    if group_ind >= len(track_groups) -1:
        for track in track_group:
            if type(track['track']) == list:
                track['track'] = np.stack(track['track'])
                track['pos_index'] = np.stack(track['pos_index'])
                if 'size' in track:
                    track['size'] = np.stack(track['size'])
            all_tracks.append(track)
        total_tracks += len(track_group)
        break
    total_tracks += len(track_group)
    for track_ind, track in enumerate(track_group):
        duplicate = False
        if track['first_frame'] > first_overlap_frames[first_group + group_ind]:
            looking = True
            for alt_track in track_groups[first_group + group_ind + 1]:
                if type(track['track']) == list:
                    issue += 1
                if alt_track['first_frame'] == track['first_frame']:
                    if np.all(alt_track['track'][0] == track['track'][0]):
                        if type(alt_track['track']) == list:
                            alt_track['track'] = np.stack(alt_track['track'])
                            alt_track['pos_index'] = np.stack(alt_track['pos_index'])
                            if 'size' in alt_track:
                                alt_track['size'] = np.stack(alt_track['size'])
                        count += 1
                        track_pairs.append([track, alt_track])
                        looking = False
                        duplicate = True
                        break
            if looking:
                not_found.append(track)
        if not duplicate:
            all_tracks.append(track)
                
print(f'{len(track_pairs)} track pairs. {len(not_found)} unmatched.')

all_tracks_file = os.path.join(observation_folder, 'raw_tracks.npy')
np.save(all_tracks_file, all_tracks)

In [ ]:
print(issue)

In [ ]:
print(total_tracks, len(track_pairs) + len(all_tracks))
print(len(all_tracks))

In [ ]:
all_tracks_file = os.path.join(observation_folder, 'raw_tracks.npy')
np.save(all_tracks_file, all_tracks)

In [ ]:
off = 0
for t in not_found:
    if len(t['track']) != 1:
        off += 1
print(off)

In [ ]:
inds_off = []
for track_ind, track in enumerate(all_tracks):
    if type(track['track']) == list:
        inds_off.append(track_ind)

In [ ]:
inds_off = []
for track_ind, track in enumerate(track_pairs):
    if type(track[1]['track']) == list:
        inds_off.append(track_ind)

In [ ]:
len(inds_off)

In [ ]:
drawn = 0
max_draw = 10
for pair in track_pairs[5000:]:
    if len(pair[0]['track']) != len(pair[1]['track']):
        print(len(pair[0]['track']), len(pair[1]['track']))
        if drawn < max_draw:
            plt.figure()
            plt.scatter(pair[0]['track'][:,0], pair[0]['track'][:,1], s=50)
            plt.scatter(np.stack(pair[1]['track'])[:,0], np.stack(pair[1]['track'])[:,1], s=10, c='r')
            drawn += 1

In [ ]:
track_pairs[-1][0]['last_frame']

In [ ]:
frame_files = os.path.join(observation_folder, 'example-frames', '*.jpg')
frame_files = sorted(glob.glob(frame_files), key=lambda f: int(f.split('.')[-2].split('_')[-1]))

In [ ]:
import matplotlib.pyplot as plt

example_ind = 24
frame_file = frame_files[example_ind]
obs_ind = int(frame_file.split('.')[-2].split('_')[-1])
plt.figure(figsize=(20,20))
plt.imshow(plt.imread(frame_file))
plt.scatter(48+centers[obs_ind][:, 0], 48+centers[obs_ind][:, 1])

In [ ]:
points = [c.shape[0] for c in centers]

In [ ]:
first_issue_frame = 17718
last_issue_frame = 32659

In [ ]:
ff = 17719
x=np.argmin(points[ff:ff+15000])+ff

In [ ]:
plt.plot(points[first_issue_frame-1:last_issue_frame+2], marker='.')

In [ ]:
centers_file = os.path.join(observation_folder, 'centers.npy')

In [ ]:
centers = np.load(centers_file, allow_pickle=True)

In [ ]:
for ind in range(first_issue_frame, last_issue_frame + 1):
    centers[ind] = np.copy(centers[5543])


In [ ]:
centers_shape = [c.shape[0] for c in centers]

In [ ]:
plt.plot(centers_shape)

In [ ]:
np.save(centers_file, centers)